In [6]:
import openai
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# First stage: Input
prompt = input("What would you like your playlist to be about? ")
num_songs = int(input("How many songs do you want on your playlist? "))
playlist_name = input("What do you want to name your playlist? ")

scope = "playlist-modify-public"  # define scope before initializing SpotifyOAuth
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='', client_secret='', redirect_uri='', scope=scope))

# Second stage: Song picking
openai.api_key = "" # Replace with your OpenAI API key

song_list = []
while len(song_list) < num_songs:
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"Generate a song about {prompt}",
        max_tokens=60,
        n=1,
        stop=None,
        temperature=0.7
    )

    song_title = response.choices[0].text.strip()
    if song_title not in song_list:
        song_list.append(song_title)

# Third stage: Spotify integration

# Check if playlist already exists
existing_playlists = sp.current_user_playlists()
playlist_exists = False
playlist_id = None
for playlist in existing_playlists["items"]:
    if playlist["name"] == playlist_name:
        playlist_exists = True
        playlist_id = playlist["id"]
        break

# Create a new playlist if it doesn't exist
if not playlist_exists:
    sp.user_playlist_create(user=sp.current_user()["id"], name=playlist_name, public=True)
    existing_playlists = sp.current_user_playlists()
    for playlist in existing_playlists["items"]:
        if playlist["name"] == playlist_name:
            playlist_id = playlist["id"]
            break

# Add songs to the playlist
for song_title in song_list:
    results = sp.search(q=song_title, limit=1, type="track")
    if results["tracks"]["items"]:
        track_id = results["tracks"]["items"][0]["id"]
        sp.playlist_add_items(playlist_id, [track_id])
        print(f"Added {song_title} to the playlist.")
    else:
        print(f"Could not find a song for {song_title}.")


SpotifyOauthError: error: invalid_client, error_description: Invalid client